In [ ]:
import os
from llama_index.llms.cohere import Cohere
from llama_index.core import Settings
from llama_index.core import GraphIndex
from llama_index.core.prompts import PromptTemplate
from llama_index.graph_stores.neo4j import Neo4jGraphStore
from neo4j import GraphDatabase
from llama_index.core.tools import FunctionTool
from dotenv import load_dotenv
from  import tokenizer

In [ ]:
load_dotenv()

Settings.llm = Cohere(api_key=os.getenv("COHERE_API_KEY"), model="command-r", temperature=0.2)

In [ ]:
system_prompt = PromptTemplate(
"""
Eres un psiquiatra clínico profesional.
Reglas:

1. Nunca realices un diagnóstico sin antes llamar a la herramienta BERT.
2. Si falta información para diagnosticar, solicita más datos antes.
3. Cuando tengas un diagnóstico ICD:
    - Consulta Neo4J para obtener evidencia real del dataset.
4. Nunca inventes diagnósticos ni medicación.
5. Responde siempre con el siguiente formato:
6. Si el usuario describe síntomas, siempre debes llamar primero a la herramienta BERT para obtener el ICD.

Formato de respuesta:
- Resumen del caso
- Diagnóstico preliminar
- Evidencia clínica encontrada
- Información faltante
- Probabilidad de diagnóstico psiquiatrico segun el resultado de BERT
- Próximo paso recomendado
"""
)

In [ ]:
def bert_diagnose(text: str) -> dict:
    """
    Ejecuta el modelo BERT sobre un texto clínico y devuelve un ICD-10.
    """
    # 1️⃣ Tokenizar
    inputs = tokenizer(
        text,
        return_tensors="pt",
        truncation=True,
        padding=True
    )

    # 2️⃣ Pasar por el modelo
    outputs = bert_model(**inputs)
    probs = torch.softmax(outputs.logits, dim=1)[0]

    # 3️⃣ Obtener predicción
    pred_idx = torch.argmax(probs).item()
    confidence = probs[pred_idx].item()

    icd_code = id_to_icd[pred_idx]     # dict { 0:"F20.0", 1:"F20.1" ...}
    icd_label = icd_names[pred_idx]    # dict con nombres legibles

    return {
        "icd_code": icd_code,
        "label": icd_label,
        "confidence": float(confidence)
    }

bert_tool = FunctionTool.from_defaults(
    name="bert_classifier",
    description="Saca las probabilidades del paciente de padecer un trastorno psiquiátrico dado su historia clínica.",
    func=bert_diagnose
)


In [ ]:
load_dotenv()

driver = GraphDatabase.driver(
    os.getenv("NEO4J_URI"),
    auth=(os.getenv("NEO4J_USER"), os.getenv("NEO4J_PASS"))
)

def query_neo4j(diagnostico: str) -> dict:
    """
    Dado un código ICD, recupera casos reales del grafo Neo4J.
    """
    query = """
    MATCH (d:Diagnostico {terminoEN: $terminoEN})<-[:`DIAGNOSTICO_ASOCIADO`]-(p:Paciente)
    RETURN p.numero_historia AS paciente_id, d.terminoEN AS diagnostico, d.ICD10 as icd_code
    LIMIT 25
    """

    with driver.session() as session:
        results = session.run(query, terminoEN=diagnostico)

        data = [
            {
                "paciente_id": record["paciente_id"],
                "diagnostico": record["diagnostico"],
                "icd_code": record["icd_code"]
            }
            for record in results
        ]

    return {
        "diagnostico": diagnostico,
        "n_casos": len(data),
        "ejemplos": data
    }

neo4j_tool = FunctionTool.from_defaults(
    name="neo4j_lookup",
    description="Consulta datos reales del dataset en Neo4J usando el nombre del diagnóstico en español.",
    func=query_neo4j
)


In [ ]:


graph_store = Neo4jGraphStore(
    url="bolt://localhost:7687",
    username="neo4j",
    password="tu_password"
)


In [ ]:
graph_index = GraphIndex.from_graph_store(graph_store)

In [ ]:
query_engine = graph_index.as_query_engine(
    text_qa_template=system_prompt,
    tools=[bert_tool, neo4j_tool],
    enforce_execution=True
)

Settings.callback_manager.add_open_trace()

In [ ]:
response = query_engine.query(
    "¿Qué diagnósticos previos tiene el paciente 123?"
)

print(response)

In [ ]:
response = query_engine.query(
    "Resumen clínico del paciente 501"
)

print(response)

In [ ]:
response = query_engine.query(
    "Paciente dice que escucha voces y siente que alguien controla sus pensamientos."
)

print(response)